In [2]:
import numpy as np
import pandas as pd
import datetime
import pickle #pickle序列化对象并保存到磁盘中，并在需要的时候读取出来
import os
import re
import catboost as cb
from catboost import CatBoostClassifier
import xgboost as xgb
import sklearn

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
def load_data():
    with open("clean_data_without_no_buy.pkl", "rb") as f:
        df_user, df_sku, df_action = pickle.load(f)
    return df_user, df_sku, df_action
df_user, df_sku, df_action = load_data()

In [4]:
#获取从某个时间段内的总的用户行为特征
def get_user_action_feature_from_to(start_time, end_time, df_action):
    df_user_action = df_action.loc[(df_action.loc[:, "time"] >= start_time) & (df_action.loc[:, "time"] < end_time)]
    df_user_action = df_user_action.loc[:, ["user_id", "type", "time"]]
#     print(df_user_action.head())
    df_user_action_type_cnt = df_user_action.groupby(["user_id", "type"]).count().reset_index()
    df_user_action_type_cnt.rename(columns={"time":"cnt"}, inplace=True)
#     print(df_user_action_type_cnt.head())
    
    df_user_action_type_cnt.loc[:, "browse_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 1) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "addcart_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 2) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "delcart_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 3) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "buy_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 4) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "follow_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 5) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "click_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 6) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    
    df_user_action_type_cnt.pop("type")
    df_user_action_type_cnt.pop("cnt")
    df_user_action_type_cnt = df_user_action_type_cnt.groupby("user_id").sum().reset_index()
    
#     print(df_user_action_type_cnt.head())
    eps = 10**(-6)
    df_user_action_type_cnt.loc[:, "browse_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "browse_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "addcart_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "addcart_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "delcart_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "delcart_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "follow_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "follow_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "click_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "click_cnt"] + eps)
    
    df_user_action_type_cnt.loc[:, "browse_rate"] = (df_user_action_type_cnt.loc[:, "browse_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "addcart_rate"] = (df_user_action_type_cnt.loc[:, "addcart_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "delcart_rate"] = (df_user_action_type_cnt.loc[:, "delcart_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "follow_rate"] = (df_user_action_type_cnt.loc[:, "follow_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "click_rate"] = (df_user_action_type_cnt.loc[:, "click_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
                                              
    df_user_action_feature = df_user_action_type_cnt
    return df_user_action_feature
    
# get_user_action_feature_from_to(pd.to_datetime("2016-02-01"), pd.to_datetime("2016-04-18"), df_action)

In [5]:
#获取某时间段内用户针对某类别cate商品的行为特征
def get_user_cate_feature_from_to(start_time, end_time, df_action, df_sku):
    df_user_action = df_action.loc[(df_action.loc[:, "time"] >= start_time) & (df_action.loc[:, "time"] < end_time)]
    df_user_action = df_user_action.loc[:, ["user_id", "sku_id", "type", "time"]]
    df_user_action = pd.merge(df_user_action, df_sku.loc[:, ["sku_id", "cate"]], how="left", on="sku_id")
    df_user_action.pop("sku_id")
    df_user_action_type_cnt = df_user_action.groupby(["user_id", "cate", "type"]).count().reset_index()
    df_user_action_type_cnt.rename(columns={"time":"cnt"}, inplace=True)
#     print(df_user_action_type_cnt.head())
    
    df_user_action_type_cnt.loc[:, "browse_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 1) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "addcart_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 2) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "delcart_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 3) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "buy_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 4) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "follow_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 5) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "click_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 6) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    
    df_user_action_type_cnt.pop("type")
    df_user_action_type_cnt.pop("cnt")
    df_user_action_type_cnt = df_user_action_type_cnt.groupby(["user_id", "cate"]).sum().reset_index()
    
#     print(df_user_action_type_cnt.head())
    eps = 10**(-6)
    df_user_action_type_cnt.loc[:, "browse_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "browse_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "addcart_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "addcart_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "delcart_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "delcart_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "follow_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "follow_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "click_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "click_cnt"] + eps)
    
    df_user_action_type_cnt.loc[:, "browse_rate"] = (df_user_action_type_cnt.loc[:, "browse_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "addcart_rate"] = (df_user_action_type_cnt.loc[:, "addcart_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "delcart_rate"] = (df_user_action_type_cnt.loc[:, "delcart_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "follow_rate"] = (df_user_action_type_cnt.loc[:, "follow_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "click_rate"] = (df_user_action_type_cnt.loc[:, "click_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
                                              
    df_user_cate_feature = df_user_action_type_cnt
    return df_user_cate_feature
    
# get_user_cate_feature_from_to(pd.to_datetime("2016-03-01"), pd.to_datetime("2016-03-30"), df_action, df_sku)

In [6]:
#获取某时间段内用户针对某品牌brand商品的行为特征
def get_user_brand_feature_from_to(start_time, end_time, df_action, df_sku):
    df_user_action = df_action.loc[(df_action.loc[:, "time"] >= start_time) & (df_action.loc[:, "time"] < end_time)]
    df_user_action = df_user_action.loc[:, ["user_id", "sku_id", "type", "time"]]
    df_user_action = pd.merge(df_user_action, df_sku.loc[:, ["sku_id", "brand"]], how="left", on="sku_id")
    df_user_action.pop("sku_id")
    df_user_action_type_cnt = df_user_action.groupby(["user_id", "brand", "type"]).count().reset_index()
    df_user_action_type_cnt.rename(columns={"time":"cnt"}, inplace=True)
#     print(df_user_action_type_cnt.head())
    
    df_user_action_type_cnt.loc[:, "browse_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 1) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "addcart_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 2) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "delcart_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 3) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "buy_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 4) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "follow_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 5) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "click_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 6) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    
    df_user_action_type_cnt.pop("type")
    df_user_action_type_cnt.pop("cnt")
    df_user_action_type_cnt = df_user_action_type_cnt.groupby(["user_id", "brand"]).sum().reset_index()
    
#     print(df_user_action_type_cnt.head())
    eps = 10**(-6)
    df_user_action_type_cnt.loc[:, "browse_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "browse_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "addcart_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "addcart_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "delcart_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "delcart_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "follow_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "follow_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "click_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "click_cnt"] + eps)
    
    df_user_action_type_cnt.loc[:, "browse_rate"] = (df_user_action_type_cnt.loc[:, "browse_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "addcart_rate"] = (df_user_action_type_cnt.loc[:, "addcart_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "delcart_rate"] = (df_user_action_type_cnt.loc[:, "delcart_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "follow_rate"] = (df_user_action_type_cnt.loc[:, "follow_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "click_rate"] = (df_user_action_type_cnt.loc[:, "click_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
                                              
    df_user_brand_feature = df_user_action_type_cnt
    return df_user_brand_feature
    
# get_user_brand_feature_from_to(pd.to_datetime("2016-02-01"), pd.to_datetime("2016-05-02"), df_action, df_sku)

In [7]:
#获取某时间段内用户针对某商品sku_id的行为特征
def get_user_sku_feature_from_to(start_time, end_time, df_action):
    df_user_action = df_action.loc[(df_action.loc[:, "time"] >= start_time) & (df_action.loc[:, "time"] < end_time)]
    df_user_action = df_user_action.loc[:, ["user_id", "sku_id", "type", "time"]]
    df_user_action_type_cnt = df_user_action.groupby(["user_id", "sku_id", "type"]).count().reset_index()
    df_user_action_type_cnt.rename(columns={"time":"cnt"}, inplace=True)
#     print(df_user_action_type_cnt.head())
    
    df_user_action_type_cnt.loc[:, "browse_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 1) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "addcart_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 2) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "delcart_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 3) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "buy_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 4) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "follow_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 5) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "click_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 6) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    
    df_user_action_type_cnt.pop("type")
    df_user_action_type_cnt.pop("cnt")
    df_user_action_type_cnt = df_user_action_type_cnt.groupby(["user_id", "sku_id"]).sum().reset_index()
    
#     print(df_user_action_type_cnt.head())
    eps = 10**(-6)
    df_user_action_type_cnt.loc[:, "browse_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "browse_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "addcart_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "addcart_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "delcart_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "delcart_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "follow_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "follow_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "click_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "click_cnt"] + eps)
    
    df_user_action_type_cnt.loc[:, "browse_rate"] = (df_user_action_type_cnt.loc[:, "browse_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "addcart_rate"] = (df_user_action_type_cnt.loc[:, "addcart_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "delcart_rate"] = (df_user_action_type_cnt.loc[:, "delcart_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "follow_rate"] = (df_user_action_type_cnt.loc[:, "follow_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "click_rate"] = (df_user_action_type_cnt.loc[:, "click_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
                                              
    df_user_sku_feature = df_user_action_type_cnt
    return df_user_sku_feature
    
# get_user_sku_feature_from_to(pd.to_datetime("2016-03-01"), pd.to_datetime("2016-03-02"), df_action)

In [8]:
#获取商品某时间段内被用户行为的特征
def get_sku_action_feature_from_to(start_time, end_time, df_action):
    df_user_action = df_action.loc[(df_action.loc[:, "time"] >= start_time) & (df_action.loc[:, "time"] < end_time)]
    df_user_action = df_user_action.loc[:, ["sku_id", "type", "time"]]
    df_user_action_type_cnt = df_user_action.groupby(["sku_id", "type"]).count().reset_index()
    df_user_action_type_cnt.rename(columns={"time":"cnt"}, inplace=True)
#     print(df_user_action_type_cnt.head())
    
    df_user_action_type_cnt.loc[:, "browse_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 1) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "addcart_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 2) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "delcart_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 3) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "buy_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 4) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "follow_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 5) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    df_user_action_type_cnt.loc[:, "click_cnt"] = ((df_user_action_type_cnt.loc[:, "type"] == 6) \
                                            * df_user_action_type_cnt.loc[:, "cnt"]).astype(np.float32)
    
    df_user_action_type_cnt.pop("type")
    df_user_action_type_cnt.pop("cnt")
    df_user_action_type_cnt = df_user_action_type_cnt.groupby("sku_id").sum().reset_index()
    
#     print(df_user_action_type_cnt.head())
    eps = 10**(-6)
    df_user_action_type_cnt.loc[:, "browse_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "browse_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "addcart_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "addcart_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "delcart_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "delcart_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "follow_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "follow_cnt"] + eps)
    df_user_action_type_cnt.loc[:, "click_rate"] = df_user_action_type_cnt.loc[:, "buy_cnt"] \
                                                    / (df_user_action_type_cnt.loc[:, "click_cnt"] + eps)
    
    df_user_action_type_cnt.loc[:, "browse_rate"] = (df_user_action_type_cnt.loc[:, "browse_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "addcart_rate"] = (df_user_action_type_cnt.loc[:, "addcart_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "delcart_rate"] = (df_user_action_type_cnt.loc[:, "delcart_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "follow_rate"] = (df_user_action_type_cnt.loc[:, "follow_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
    df_user_action_type_cnt.loc[:, "click_rate"] = (df_user_action_type_cnt.loc[:, "click_rate"] \
                                            .apply(lambda x : 1.0 if x>0.9999 else x)).astype(np.float32)
                                              
    df_sku_action_feature = df_user_action_type_cnt
    return df_sku_action_feature
    
# get_sku_action_feature_from_to(pd.to_datetime("2016-03-01"), pd.to_datetime("2016-03-30"), df_action)

In [9]:
#一个辅助函数，用来命名多个可能重名的属性特征
def rename_columns_with_day(df, fixstr):
    column_dict = {"browse_cnt" : "browse_cnt"+fixstr, \
                  "addcart_cnt" : "addcart_cnt"+fixstr, \
                  "delcart_cnt" : "delcart_cnt"+fixstr, \
                  "buy_cnt" : "buy_cnt"+fixstr, \
                  "follow_cnt" : "follow_cnt"+fixstr, \
                  "click_cnt" : "click_cnt"+fixstr, \
                  "browse_rate" : "browse_rate"+fixstr, \
                  "addcart_rate" : "addcart_rate"+fixstr, \
                  "delcart_rate" : "delcart_rate"+fixstr, \
                  "follow_rate" : "follow_rate"+fixstr, \
                  "click_rate" : "click_rate"+fixstr}
    df.rename(columns=column_dict, inplace=True)
    return df

In [10]:
#获取用户的固有特征（基本特征，整个时间段的行为特征）
def get_user_constant_feature(df_user, df_sku, df_action):
    
    df_user_base_feature = df_user.copy()
    
    start_time = pd.to_datetime("2016-01-01")
    end_time = pd.to_datetime("2016-06-07")
    df_user_action_feature_all_days = get_user_action_feature_from_to(start_time, end_time, df_action)
    df_user_cate_feature_all_days = get_user_cate_feature_from_to(start_time, end_time, df_action, df_sku)
    df_user_brand_feature_all_days = get_user_brand_feature_from_to(start_time, end_time, df_action, df_sku)
    df_user_sku_feature_all_days = get_user_sku_feature_from_to(start_time, end_time, df_action)
    
    df_user_action_feature_all_days = rename_columns_with_day(df_user_action_feature_all_days, "_user_all")
    df_user_cate_feature_all_days = rename_columns_with_day(df_user_cate_feature_all_days, "_user_cate_all")
    df_user_brand_feature_all_days = rename_columns_with_day(df_user_brand_feature_all_days, "_user_brand_all")
    df_user_sku_feature_all_days = rename_columns_with_day(df_user_sku_feature_all_days, "_user_sku_all")
    
    list_df_user_constant_feature = [df_user_base_feature, \
                                    df_user_action_feature_all_days, \
                                    df_user_cate_feature_all_days, \
                                    df_user_brand_feature_all_days, \
                                    df_user_sku_feature_all_days]
    
    return list_df_user_constant_feature

list_df_user_constant_feature = get_user_constant_feature(df_user, df_sku, df_action)

In [11]:
#获取商品的固有特征（基本特征，整个时间段的被用户行为特征）
def get_sku_constant_feature(df_sku, df_action):
    df_sku_base_feature = df_sku
    df_sku_action_feature_all_days = get_sku_action_feature_from_to(pd.to_datetime("2016-01-01"), pd.to_datetime("2016-06-07"), df_action)
    df_sku_action_feature_all_days = rename_columns_with_day(df_sku_action_feature_all_days, "_sku_all")
    list_df_sku_constant_feature = [df_sku_base_feature, df_sku_action_feature_all_days]
    return list_df_sku_constant_feature
list_df_sku_constant_feature = get_sku_constant_feature(df_sku, df_action)

In [12]:
#获取一个时间窗口的购买前多个不同时间段的特征    
def get_feature_of_one_window(start_time, end_time, df_action, df_sku):
    if end_time - start_time < datetime.timedelta(days=30):
#     if end_time - start_time < datetime.timedelta(days=30):
        raise("时间区间太短啦！")
    #统计时间段特征使用 [1, 2, 3, 5, 7, 10, 15, 20, 30] 天的
    before_days = [1, 2, 3, 5, 7, 10, 15, 20, 30]
#     before_days = [1, 2]
    
    list_df_user_action_feature_before_days = [] #用户对所有商品行为特征
    list_df_user_cate_feature_before_days = [] #用户对不同类商品别行为特征
    list_df_user_brand_feature_before_days = [] #用户对不同品牌商品行为特征
    list_df_user_sku_feature_before_days = [] #用户对不同商品行为特征
    list_df_sku_action_feature_before_days = [] #商品被所有用户行为的特征
    
    for day in before_days:
        start_before_day = end_time - datetime.timedelta(days=day)
        
        df_user_action_feature = get_user_action_feature_from_to(start_before_day, end_time, df_action)
        df_user_cate_feature = get_user_cate_feature_from_to(start_before_day, end_time, df_action, df_sku)
        df_user_brand_feature = get_user_brand_feature_from_to(start_before_day, end_time, df_action, df_sku)
        df_user_sku_feature = get_user_sku_feature_from_to(start_before_day, end_time, df_action)
        df_sku_action_feature = get_sku_action_feature_from_to(start_before_day, end_time, df_action)
        
        df_user_action_feature = rename_columns_with_day(df_user_action_feature, "_user_"+str(day))
        df_user_cate_feature = rename_columns_with_day(df_user_cate_feature, "_user_cate_"+str(day))
        df_user_brand_feature = rename_columns_with_day(df_user_brand_feature, "_user_brand_"+str(day))
        df_user_sku_feature = rename_columns_with_day(df_user_sku_feature, "_user_sku_"+str(day))
        df_sku_action_feature = rename_columns_with_day(df_sku_action_feature, "_sku_"+str(day))
        
        
        list_df_user_action_feature_before_days.append(df_user_action_feature)
        list_df_user_cate_feature_before_days.append(df_user_cate_feature)
        list_df_user_brand_feature_before_days.append(df_user_brand_feature)
        list_df_user_sku_feature_before_days.append(df_user_sku_feature)
        list_df_sku_action_feature_before_days.append(df_sku_action_feature)
    
    
    return list_df_user_action_feature_before_days, \
            list_df_user_cate_feature_before_days, \
            list_df_user_brand_feature_before_days, \
            list_df_user_sku_feature_before_days, \
            list_df_sku_action_feature_before_days
# lists_features_one_window = get_feature_of_one_window(pd.to_datetime("2016-02-05"), pd.to_datetime("2016-02-10"), df_action, df_sku)

In [13]:
#获取用户在某时间段内是否购买的标签，令 end_time - start_time = 5，用来生成数据集label
def get_user_sku_buy_label_from_to(start_time, end_time, df_action):
    print(start_time, end_time)
    df_user_sku_feature = get_user_sku_feature_from_to(start_time, end_time, df_action)
    df_user_sku_buy_label = df_user_sku_feature.loc[:, ["user_id", "sku_id", "buy_cnt"]]
    df_user_sku_buy_label.loc[:, "buy_label"] = df_user_sku_buy_label.loc[:, "buy_cnt"] \
                                                .apply(lambda x : True if x>=1 else False) #catboost使用1和-1做二分类标签
    df_user_sku_buy_label.pop("buy_cnt")
    return df_user_sku_buy_label

# df_user_sku_buy_label = get_user_sku_buy_label_from_to(pd.to_datetime("2016-02-05"), pd.to_datetime("2016-02-06"), df_action)
# print(df_user_sku_buy_label.loc[df_user_sku_buy_label.loc[:, "buy_label"] == True])

In [14]:
def fix_dataset(df_dataset):
    column_list = df_dataset.columns.values.tolist()
#     print(column_list)
    df_dataset.fillna(0, inplace=True)

    for column in column_list:
        if re.match(r"user_id[0-9]", column) or re.match(r"sku_id[0-9]", column) is not None:
            df_dataset.pop(column)

        if re.match(r"cate[0-9]", column) or re.match(r"brand[0-9]", column) is not None:
            df_dataset.pop(column)

#         if re.match(r".*_cnt.*", column) is not None:
#             df_dataset.loc[:, column] = df_dataset.loc[:, column].astype(np.int16)

#         if re.match(r".*_rate.*", column) is not None:
#             df_dataset.loc[:, column] = df_dataset.loc[:, column].astype(np.float16)

    return df_dataset


In [15]:
#获取一个时间窗口的训练集
def get_dataset_of_one_window(start_time, end_time, \
                                    df_user, df_sku, df_action, \
                                    list_df_user_constant_feature, list_df_sku_constant_feature):
    list_df_user_action_feature_before_days, \
    list_df_user_cate_feature_before_days, \
    list_df_user_brand_feature_before_days, \
    list_df_user_sku_feature_before_days, \
    list_df_sku_action_feature_before_days = get_feature_of_one_window(start_time, end_time, df_action, df_sku)
    
    df_user_sku_buy_label = get_user_sku_buy_label_from_to(end_time, end_time + datetime.timedelta(days=5), df_action)
    
    df_dataset = list_df_user_sku_feature_before_days[-1].loc[:, ["user_id", "sku_id"]].drop_duplicates()
    print("len_df_dataset: ", len(df_dataset))
    df_dataset = pd.merge(df_dataset, df_sku.loc[:, ["sku_id", "cate", "brand"]], how="left", on="sku_id")
    
    suffix_y = 1
    for df_user_constant_feature in list_df_user_constant_feature:
        if "sku_id" in df_user_constant_feature.columns.tolist():
            df_dataset = pd.merge(df_dataset, df_user_constant_feature, how="left", on=["user_id", "sku_id"], suffixes=("", str(suffix_y)), sort=False)
        elif "cate" in df_user_constant_feature.columns.tolist():
            df_dataset = pd.merge(df_dataset, df_user_constant_feature, how="left", on=["user_id", "cate"], suffixes=("", str(suffix_y)), sort=False)
        elif "brand" in df_user_constant_feature.columns.tolist():
            df_dataset = pd.merge(df_dataset, df_user_constant_feature, how="left", on=["user_id", "brand"], suffixes=("", str(suffix_y)), sort=False)
        else:
            df_dataset = pd.merge(df_dataset, df_user_constant_feature, how="left", on="user_id", suffixes=("", str(suffix_y)), sort=False)
        
        df_user_constant_feature = None
#         print(1)
        suffix_y += 1
        
    for df_sku_constant_feature in list_df_sku_constant_feature:
        df_dataset = pd.merge(df_dataset, df_sku_constant_feature, how="left", on="sku_id", suffixes=("", str(suffix_y)), sort=False)
        df_sku_constant_feature = None
#         print(2)
        suffix_y += 1
        
    for df_user_action_feature_before_days in list_df_user_action_feature_before_days:
        df_dataset = pd.merge(df_dataset, df_user_action_feature_before_days, how="left", on="user_id", suffixes=("", str(suffix_y)), sort=False)
        df_user_action_feature_before_days = None
#         print(3)
        suffix_y += 1
        
    for df_user_cate_feature_before_days in list_df_user_cate_feature_before_days:
        df_dataset = pd.merge(df_dataset, df_user_cate_feature_before_days, how="left", on=["user_id", "cate"], suffixes=("", str(suffix_y)), sort=False)
        df_user_cate_feature_before_days = None
#         print(4)
        suffix_y += 1
        
    for df_user_brand_feature_before_days in list_df_user_brand_feature_before_days:
        df_dataset = pd.merge(df_dataset, df_user_brand_feature_before_days, how="left", on=["user_id", "brand"], suffixes=("", str(suffix_y)), sort=False)
        df_user_brand_feature_before_days = None
#         print(5)
        suffix_y += 1
        
    for df_user_sku_feature_before_days in list_df_user_sku_feature_before_days:
        df_dataset = pd.merge(df_dataset, df_user_sku_feature_before_days, how="left", on=["user_id", "sku_id"], suffixes=("", str(suffix_y)), sort=False)
        df_user_sku_feature_before_days = None
#         print(6)
        suffix_y += 1
        
    for df_sku_action_feature_before_days in list_df_sku_action_feature_before_days:
        df_dataset = pd.merge(df_dataset, df_sku_action_feature_before_days, how="left", on="sku_id", suffixes=("", str(suffix_y)), sort=False)
        df_sku_action_feature_before_days = None
#         print(7)
        suffix_y += 1
        
    df_dataset = pd.merge(df_dataset, df_user_sku_buy_label, how="left", on=["user_id", "sku_id"], suffixes=("", str(suffix_y)), sort=False)
    df_dataset.loc[:, "buy_label"] = df_dataset.loc[:, "buy_label"].apply(lambda x: False if np.isnan(x) else x)
    
    df_dataset = fix_dataset(df_dataset)
    print("len_df_dataset: ", len(df_dataset))
    return df_dataset

# df_dataset = get_dataset_of_one_window(pd.to_datetime("2016-02-05"), pd.to_datetime("2016-02-07"), \
#                                     df_user, df_sku, df_action, \
#                                     list_df_user_constant_feature, list_df_sku_constant_feature)
# df_dataset

In [16]:
#获取数据集
def get_dataset(window_size, window_stride, whole_start_time, whole_end_time, \
                df_user, df_sku, df_action, \
                list_df_user_constant_feature, list_df_sku_constant_feature):
    df_dataset = pd.DataFrame()
    start_time = whole_start_time
    end_time = start_time + datetime.timedelta(days=window_size)
    while end_time < whole_end_time:
        one_window_dataset = get_dataset_of_one_window(start_time, end_time, \
                                    df_user, df_sku, df_action, \
                                    list_df_user_constant_feature, list_df_sku_constant_feature)
        
        df_dataset = one_window_dataset.append(df_dataset)
        
        start_time = start_time + datetime.timedelta(days=window_stride)
        end_time = start_time + datetime.timedelta(days=window_size)
        
    return df_dataset

df_dataset = get_dataset(30, 10, pd.to_datetime("2016-02-01"), pd.to_datetime("2016-04-15"), \
                         df_user, df_sku, df_action, \
                         list_df_user_constant_feature, list_df_sku_constant_feature)

2016-03-02 00:00:00 2016-03-07 00:00:00
len_df_dataset:  108712
len_df_dataset:  108712
2016-03-12 00:00:00 2016-03-17 00:00:00
len_df_dataset:  181605
len_df_dataset:  181605
2016-03-22 00:00:00 2016-03-27 00:00:00
len_df_dataset:  247687
len_df_dataset:  247687
2016-04-01 00:00:00 2016-04-06 00:00:00
len_df_dataset:  259650
len_df_dataset:  259650
2016-04-11 00:00:00 2016-04-16 00:00:00
len_df_dataset:  254062
len_df_dataset:  254062


In [17]:
def save_df_dataset(df_dataset):
    if not os.path.exists("df_dataset.pkl"):
        with open("df_dataset.pkl", "wb") as f:
            pickle.dump(df_dataset, f)
            
def load_df_dataset():
    with open("df_dataset.pkl", "rb") as f:
        df_dataset = pickle.load(f)
    return df_dataset

# save_df_dataset(df_dataset)
# df_dataset = load_df_dataset()

In [18]:
def get_train_test_dataset(df_dataset, test_size):
    train_size = len(df_dataset) - test_size
    train_dataset = df_dataset.head(train_size)
    test_dataset = df_dataset.tail(test_size)
    return train_dataset, test_dataset

df_train_dataset, df_test_dataset = get_train_test_dataset(df_dataset, 254062)

In [46]:
#进行训练，使用catboost算法
def train(df_dataset, iteration, model_path):
    train_data = df_dataset.copy()
    train_data.pop("buy_label")
    train_data.pop("user_id")
    train_data.pop("sku_id")
    print(len(train_data.columns.tolist()))
    
    cat_feature_list = ["cate", "brand", "age", "sex", "a1", "a2", "a3"]
    cat_feature_list = [feat for feat in cat_feature_list if feat in train_data.columns.tolist()]
    
    train_label = df_dataset.loc[:, "buy_label"].apply(lambda x: 1 if x else 0)

    model = CatBoostClassifier(iterations=iteration, \
                               loss_function="Logloss", \
#                                learning_rate=1, \
                               task_type="GPU", \
                              depth=6, \
                              leaf_estimation_method="Newton", \
                               one_hot_max_size=2)
    model.fit(train_data, train_label, cat_features=cat_feature_list)
    with open(model_path, "wb") as f:
        pickle.dump(model, f)
    return model

# print(df_dataset.loc[:, "buy_label"])
train(df_train_dataset, 150, "model.cb")

0:	learn: 0.6000216	total: 213ms	remaining: 31.7s
1:	learn: 0.5200421	total: 416ms	remaining: 30.8s
2:	learn: 0.4497099	total: 618ms	remaining: 30.3s
3:	learn: 0.3905455	total: 782ms	remaining: 28.5s
4:	learn: 0.3042123	total: 961ms	remaining: 27.9s
5:	learn: 0.2456926	total: 1.14s	remaining: 27.4s
6:	learn: 0.1904799	total: 1.31s	remaining: 26.7s
7:	learn: 0.1519922	total: 1.52s	remaining: 27s
8:	learn: 0.1192004	total: 1.7s	remaining: 26.6s
9:	learn: 0.0932852	total: 1.86s	remaining: 26.1s
10:	learn: 0.0738627	total: 2.04s	remaining: 25.8s
11:	learn: 0.0591132	total: 2.23s	remaining: 25.6s
12:	learn: 0.0467990	total: 2.39s	remaining: 25.1s
13:	learn: 0.0379000	total: 2.56s	remaining: 24.9s
14:	learn: 0.0312534	total: 2.75s	remaining: 24.8s
15:	learn: 0.0258914	total: 2.94s	remaining: 24.6s
16:	learn: 0.0213190	total: 3.11s	remaining: 24.3s
17:	learn: 0.0178654	total: 3.29s	remaining: 24.1s
18:	learn: 0.0152011	total: 3.48s	remaining: 24s
19:	learn: 0.0129997	total: 3.65s	remaining: 2

In [19]:
#测试打分
def get_test_score(test_result):
    """
    test_result is a DataFrame with columns: user_id, sku_id, buy_label, predict_label
    """
    df_check_user_id = test_result.groupby("user_id").sum().reset_index()
    user_tp_cnt = len(df_check_user_id.loc[(df_check_user_id.loc[:, "buy_label"]>0) & (df_check_user_id.loc[:, "predict_label"]>0)])
    user_fp_cnt = len(df_check_user_id.loc[(df_check_user_id.loc[:, "buy_label"]==0) & (df_check_user_id.loc[:, "predict_label"]>0)])
    user_fn_cnt = len(df_check_user_id.loc[(df_check_user_id.loc[:, "buy_label"]>0) & (df_check_user_id.loc[:, "predict_label"]==0)])
    user_tn_cnt = len(df_check_user_id.loc[(df_check_user_id.loc[:, "buy_label"]==0) & (df_check_user_id.loc[:, "predict_label"]==0)])
    user_precise = user_tp_cnt/(user_tp_cnt+user_fp_cnt)
    user_recall = user_tp_cnt/(user_tp_cnt+user_fn_cnt)
    f11 = 6*user_recall*user_precise/(5*user_recall + user_precise)
    
    sku_tp_cnt = len(test_result.loc[(test_result.loc[:, "buy_label"]>0) & (test_result.loc[:, "predict_label"]>0)])
    sku_fp_cnt = len(test_result.loc[(test_result.loc[:, "buy_label"]==0) & (test_result.loc[:, "predict_label"]>0)])
    sku_fn_cnt = len(test_result.loc[(test_result.loc[:, "buy_label"]>0) & (test_result.loc[:, "predict_label"]==0)])
    sku_tn_cnt = len(test_result.loc[(test_result.loc[:, "buy_label"]==0) & (test_result.loc[:, "predict_label"]==0)])
    sku_precise = sku_tp_cnt/(sku_tp_cnt+sku_fp_cnt)
    sku_recall = sku_tp_cnt/(sku_tp_cnt+sku_fn_cnt)
    f12 = 5*sku_recall*sku_precise/(2*sku_recall + 3*sku_precise)
    
    print("user_tp_cnt", user_tp_cnt)
    print("user_fp_cnt", user_fp_cnt)
    print("user_fn_cnt", user_fn_cnt)
    print("user_tn_cnt", user_tn_cnt)
    print("user_precise", user_precise)
    print("user_recall", user_recall)
    print("f11", f11)
    
    print("sku_tp_cnt", sku_tp_cnt)
    print("sku_fp_cnt", sku_fp_cnt)
    print("sku_fn_cnt", sku_fn_cnt)
    print("sku_tn_cnt", sku_tn_cnt)
    print("sku_precise", sku_precise)
    print("sku_recall", sku_recall)
    print("f12", f12)
    
    score = 0.4*f11 + 0.6*f12
    
    return score

In [42]:
#进行测试
def test(df_dataset, model_path, threshold=0.5):
    test_result = df_dataset.loc[:, ["user_id", "sku_id", "buy_label"]].reset_index()
    test_data = df_dataset.copy()
    test_data.pop("buy_label")
    test_data.pop("user_id")
    test_data.pop("sku_id")
    
    with open(model_path, "rb") as f:
        model = pickle.load(f)
#     predict_label = model.predict(test_data)
    predict_prob = model.predict(test_data, prediction_type='Probability')
    predict_prob = predict_prob.T[1].T
    predict_label = [prob > threshold for prob in predict_prob]
    
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(test_result.loc[:, "buy_label"].values.tolist(), predict_prob)

    line = ""
    for i in range(len(thresholds)):
        line += str(fpr[i])+"\t"+str(tpr[i])+"\t"+str(thresholds[i])+"\n"
    f = open("roc_curve", "w")
    f.write(line)
    f.close()
    
    auc_score = sklearn.metrics.roc_auc_score(test_result.loc[:, "buy_label"].values.tolist(), predict_prob)
    print("auc_score: ", auc_score)
    
    test_result.loc[:, "predict_label"] = pd.Series(predict_label).T.astype(np.int32)

    score = get_test_score(test_result)
    
    return score

test(df_test_dataset, "model.cb", 0.25)

auc_score:  0.9981131468400417
user_tp_cnt 758
user_fp_cnt 580
user_fn_cnt 219
user_tn_cnt 17600
user_precise 0.5665171898355755
user_recall 0.7758444216990789
f11 0.5931916003652015
sku_tp_cnt 721
sku_fp_cnt 720
sku_fn_cnt 259
sku_tn_cnt 252362
sku_precise 0.5003469812630118
sku_recall 0.7357142857142858
f12 0.6192030230161457


0.608798453955768

In [57]:
################## train and test using xgboost ########################

In [20]:
# 由于xgboost不原生支持类别特征，将类别特征转换为 OneHot 编码
def one_hot_dataset(df_dataset):
    cat_feature_list = ["cate", "brand", "age", "sex", "a1", "a2", "a3"]
    cat_feature_list = [feat for feat in cat_feature_list if feat in df_dataset.columns.tolist()]
    df_one_hot_dataset = pd.get_dummies(df_dataset, columns=cat_feature_list)
    return df_one_hot_dataset

df_one_hot_dataset = one_hot_dataset(df_dataset)
# print(df_one_hot_dataset.head())

In [21]:
df_one_hot_train_dataset, df_ont_hot_test_dataset = get_train_test_dataset(df_one_hot_dataset, 254062)

In [25]:
def xgb_train(df_dataset, model_path):
    train_data = df_dataset.copy() # train_data 是训练集特征数据
    train_data.pop("buy_label") # 特征数据不包含标签，删除数据标签
    train_data.pop("user_id") # 用户id不应该作为特征
    train_data.pop("sku_id") # 商品id不应该作为特征
    print(len(train_data.columns.tolist()))
    
    # train_label 是训练集的数据标签
    train_label = df_dataset.loc[:, "buy_label"].apply(lambda x:1 if x else 0)
    
    # 将Pandas格式的特征及标签转换为xgboost的数据格式
    dtrain = xgb.DMatrix(train_data, label=train_label)
    
    # 设置模型参数
    xgb_param = {#学习速率
                 "learning_rate":0.3, \
                 #预剪枝机制，限制模型复杂度
                 "gamma":0.6, \
                 #每棵树的最大深度
                 "max_depth":6, \
                 #L2损失的权重，防止过拟合
                 "lambda":1, \
                 #树构建方法
                 "tree_method":"gpu_hist", \
                 #正样本权重，因为我们的类别不平衡，令正样本的权重大于一
                 "scale_pos_weight":1, \
                 #预测算法使用CPU还是GPU
                 "predictor":"gpu_predictor", \
                 #目标函数选择（二分类问题使用逻辑回归的目标函数）
                 "objective":"binary:logistic", \
                 "eval_metric":"logloss"}
    
    # 使用设定的参数及数据进行训练
    xgb_model = xgb.train(xgb_param, dtrain, num_boost_round=150, evals=[(dtrain, "train")])
    
    # 保存训练好的模型到指定的路径
    with open(model_path, "wb") as f:
        pickle.dump(xgb_model, f)
    return xgb_model

# 输入经过独热编码处理后的数据，调用训练函数进行训练
xgb_train(df_one_hot_train_dataset, "xgb_model")

614
[0]	train-logloss:0.438513
[1]	train-logloss:0.297849
[2]	train-logloss:0.209219
[3]	train-logloss:0.14993
[4]	train-logloss:0.108886
[5]	train-logloss:0.079877
[6]	train-logloss:0.059077
[7]	train-logloss:0.044042
[8]	train-logloss:0.033084
[9]	train-logloss:0.025044
[10]	train-logloss:0.019156
[11]	train-logloss:0.014819
[12]	train-logloss:0.0116
[13]	train-logloss:0.009206
[14]	train-logloss:0.007447
[15]	train-logloss:0.006128
[16]	train-logloss:0.005154
[17]	train-logloss:0.004428
[18]	train-logloss:0.003882
[19]	train-logloss:0.003454
[20]	train-logloss:0.003132
[21]	train-logloss:0.0029
[22]	train-logloss:0.002719
[23]	train-logloss:0.002595
[24]	train-logloss:0.002488
[25]	train-logloss:0.002394
[26]	train-logloss:0.002342
[27]	train-logloss:0.002258
[28]	train-logloss:0.002185
[29]	train-logloss:0.002126
[30]	train-logloss:0.002104
[31]	train-logloss:0.002063
[32]	train-logloss:0.002031
[33]	train-logloss:0.001975
[34]	train-logloss:0.001927
[35]	train-logloss:0.00191
[36]

In [26]:
def xgb_test(df_dataset, model_path, threshold=0.5):
    test_result = df_dataset.loc[:, ["user_id", "sku_id", "buy_label"]].reset_index()
    test_data = df_dataset.copy() #test_data 是验证集特征数据
    test_data.pop("buy_label") # 将验证集特征数据中的标签删掉
    test_data.pop("user_id") # 将验证集中的用户id删掉
    test_data.pop("sku_id") # 将验证集中的商品id删掉
    #将Pandas格式的验证集的特征数据转换为xgboost的数据格式
    dtest = xgb.DMatrix(test_data) 
    
    #提取用户是否购买的标签，后面与预测结果对比，得到相关评价指标
    test_label = test_result.loc[:, "buy_label"].values.tolist()
    
    #从磁盘中读取训练好的模型
    with open(model_path, "rb") as f:
        xgb_model = pickle.load(f)
    
    #利用训练好的模型进行预测，得到分类为正样本的概率值
    predict_score = xgb_model.predict(dtest)
    #根据设定好的概率阈值得到预测标签
    predict_label = [score>threshold for score in predict_score]
    
    #利用scikit-learn库中的函数，输入验证集真实标签和预测为正样本的概率，得到ROC曲线的数据
    fpr, tpr, thresholds = sklearn.metrics.roc_curve(test_label, predict_score)
    #保存ROC曲线的数据，可导出到excel中绘制ROC曲线
    f = open("roc_curve", "w")
    line = ""
    for i in range(len(thresholds)):
        line += str(fpr[i])+"\t"+str(tpr[i])+"\t"+str(thresholds[i])+"\n"
    f.write(line)
    f.close()
    
    #利用scikit-learn库中的函数，输入验证集真实标签和预测为正样本的概率，得到AUC值
    auc_score = sklearn.metrics.roc_auc_score(test_label, predict_score)
    print("auc_score: ", auc_score)
    
    #将预测标签附加到test_result中，计算F值
    test_result.loc[:, "predict_label"] = pd.Series(predict_label).T.astype(np.int32)
    score = get_test_score(test_result)
    
    return score

xgb_test(df_ont_hot_test_dataset, "xgb_model", 0.165)

auc_score:  0.9979732873543123
user_tp_cnt 794
user_fp_cnt 673
user_fn_cnt 183
user_tn_cnt 17507
user_precise 0.5412406271301977
user_recall 0.812691914022518
f11 0.5731472569778633
sku_tp_cnt 760
sku_fp_cnt 885
sku_fn_cnt 220
sku_tn_cnt 252197
sku_precise 0.46200607902735563
sku_recall 0.7755102040816326
f12 0.6099518459069021


0.5952300103352866

In [ ]:
########### 利用catboost对特征重要性进行排序 ##############

In [91]:
# 将 df 转换成 catboost 里面的 pool
def df2pool(df_dataset):
    data = df_dataset.copy()
    data.pop("buy_label")
    data.pop("user_id")
    data.pop("sku_id")
    
    cat_feature_list = ["cate", "brand", "age", "sex", "a1", "a2", "a3"]
    cat_feature_list = [feat for feat in cat_feature_list if feat in data.columns.tolist()]
    
    label = df_dataset.loc[:, "buy_label"].apply(lambda x: 1 if x else 0)
    
    data_pool = cb.Pool(data, label, cat_features=cat_feature_list)
    
    return data_pool

In [59]:
def get_feature_importance(model, df_dataset):
    train_data = df_dataset.copy()
    train_data.pop("buy_label")
    train_data.pop("user_id")
    train_data.pop("sku_id")
    
    cat_feature_list = ["cate", "brand", "age", "sex", "a1", "a2", "a3"]
    cat_feature_list = [feat for feat in cat_feature_list if feat in train_data.columns.tolist()]
    
    train_label = df_dataset.loc[:, "buy_label"].apply(lambda x: 1 if x else 0)
    
    data_pool = cb.Pool(train_data, train_label, cat_features=cat_feature_list)
    
    feature_list = train_data.columns.tolist()
    feature_importance = model.get_feature_importance(data_pool, type="FeatureImportance")
    feature_importance_list = []
    for i in range(len(feature_list)):
        feature_importance_list.append((feature_list[i], feature_importance[i]))
    
    
    return feature_importance_list

# with open("model.cb", "rb") as f:
#     model = pickle.load(f)
# feature_importance_list = get_feature_importance(model, df_train_dataset)
# fun = lambda x:-x[1]
# sorted_feature = sorted(feature_importance_list, key=fun)
# for f, ipt in sorted_feature:
#     print(f, " : ", ipt)

In [69]:
# 得到待删除的特征
def features_to_pop(feature_importance_list):
    feat_pop = []
    for f, v in feature_importance_list:
        if v < 1e-3:
            feat_pop.append(f)
    return feat_pop

feat_pop = features_to_pop(feature_importance_list)
len(feat_pop)

163

In [60]:
# 获得删除不重要特征的数据集
def get_feature_poped_dataset(df_dataset, feat_pop):
    for feat in feat_pop:
        df_dataset.pop(feat)
    return df_dataset
    
# df_feature_poped_dataset = get_feature_poped_dataset(df_dataset, feat_pop)
# print(df_feature_poped_dataset.columns.tolist())

In [73]:
df_train_dataset, df_test_dataset = get_train_test_dataset(df_feature_poped_dataset, 254062)

In [61]:
# train(df_train_dataset, 10000, "model.cb")

In [62]:
# test(df_test_dataset, "model.cb", 0.25)

In [97]:
def get_roc_curve(df_test_dataset, model_path):
    data_pool = df2pool(df_test_dataset)
    with open(model_path, "rb") as f:
        model = pickle.load(f)
    return cb.utils.get_roc_curve(model, data_pool)

fpr, tpr, thresholds = get_roc_curve(df_test_dataset, "model.cb")

f = open("roc_curve", "w")
for i in range(len(thresholds)):
    f.write(str(fpr[i]) + "\t" + str(tpr[i]) + "\t" + str(thresholds[i]) + "\n")

In [23]:
!pwd

/workspace/mnt/group/face/liulinyun/lch
